In [1]:
# Imports

# GDAX Python API: 
    # https://github.com/danpaquin/gdax-python
# GDAX API Documentation: 
    # https://docs.gdax.com
import gdax
    # Import as Git submodule
    # Note: use local install from dev branch to fix mongo connection issue
    
import time
    # Public Endpoint request limit: "3 requests per second, up to 6 requests per second in bursts."
    # https://docs.gdax.com/#rate-limits
    
# Pymongo import for connection to local client DB
from pymongo import MongoClient    

import pandas as pd
import numpy as np
import json


In [2]:
# Establish connection to GDAX public endpoint
public_client = gdax.PublicClient()

In [3]:
# Request time from GDAX server
public_client.get_time()

{'epoch': 1523761772.524, 'iso': '2018-04-15T03:09:32.524Z'}

In [ ]:
# Request orderbook at specific channel level for specified product pair
    # Level 2 request = Top 50 bids and asks, aggregated
        # Response: [price, size, number of orders]
    # Level 3 request = Full Orderbook, not aggregated
        # Response: [price, size, order_id]
        # Level 3 response returns the entire orderbook with the associated OrderID; Level 2 does not
public_client.get_product_order_book('BTC-USD', level=2)

In [ ]:
#Excessive L3 requests = reduced access/block; Add delay due to response size
time.sleep(4) 
public_client.get_product_order_book('BTC-USD', level=3)

In [ ]:
# GDAX websocket client + mongodb
    # Websocket feed for real time updating of L2 orderbook (and potential order match stream input)
# Mongo database and collection specification:
mongo_client = MongoClient('mongodb://localhost:27017/')
db = mongo_client.btcusd_db
btcusd_collection = db.btcusd_collection

In [ ]:
# Start instance of Websocketclient
    # Get Level 2 Orderbook for BTC-USD + Match Data

wsClient = gdax.WebsocketClient(url="wss://ws-feed.gdax.com",\ 
                                products=["BTC-USD"],\
                                message_type="subscribe",\
                                channels =["level2","matches"],\
                                mongo_collection=btcusd_collection,\
                                should_print=False)
time.sleep(4)

wsClient.start()

time.sleep(60)

wsClient.close()

In [ ]:
wsClient.close()